# GAN IMPLEMENTATION

***Implement GAN for Medical Dataset.***

Dataset: https://www.kaggle.com/api/v1/datasets/download/tawsifurrahman/covid19-radiography-database


<hr>

### Importing the required Modules

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import Sequence
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

<hr>

### Fetching the Data from Kaggle programmatically

In [ ]:
!curl -L -o covid19-radiography-database.zip\
  https://www.kaggle.com/api/v1/datasets/download/tawsifurrahman/covid19-radiography-database

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  778M  100  778M    0     0  20.1M      0  0:00:38  0:00:38 --:--:-- 19.6M


In [ ]:
!unzip covid19-radiography-database.zip

<hr>

### Data Preparation and Loading the Images

(Using small subset of original data due to low computational resources)

In [ ]:
dataset_path = "Covid-19 Data"
image_size = (128, 128)
batch_size = 32

- Creating a smaller subset of data

In [ ]:
def load_subset_of_images(folder, num_images=2500, image_size=(128, 128)):
    images = []
    filenames = os.listdir(folder)
    selected_files = np.random.choice(filenames, size=num_images, replace=False)
    for filename in selected_files:
        img_path = os.path.join(folder, filename)
        try:
            img = image.load_img(img_path, target_size=image_size)
            img_array = image.img_to_array(img)
            images.append(img_array)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images)

- Separating the two classes of the images

In [ ]:
num_images_per_class = 1000
covid_images = load_subset_of_images(os.path.join(dataset_path, 'COVID'), num_images=num_images_per_class)
normal_images = load_subset_of_images(os.path.join(dataset_path, 'Normal'), num_images=num_images_per_class)

- Normalization

In [ ]:
covid_images = covid_images / 255.0
normal_images = normal_images / 255.0

<hr>

### Defining the GAN Architecture

- Generator Part

In [ ]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(16 * 16 * 256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((16, 16, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))  # Upscale to (32, 32)
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))  # Upscale to (64, 64)
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh'))  # Final upscale to (128, 128)
    return model

- Discriminator Part

In [ ]:
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

- GAN Combined

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([generator, discriminator])
    return model

<hr>

### Sampling Images

In [ ]:
def sample_images():
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale back to [0, 1]

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    plt.show()

### Model Training

- Hyperparamters

In [ ]:
latent_dim = 100
input_shape = (128, 128, 3)
epochs = 100
batch_size = 32
sample_interval = 500

- Training Data

In [ ]:
real_images = np.concatenate([covid_images, normal_images])
real_images = (real_images - 0.5) * 2  # Rescale to [-1, 1]

- Model Compilation

In [ ]:
generator = build_generator(latent_dim)
discriminator = build_discriminator(input_shape)
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mean_squared_error', metrics=['accuracy'])
gan = build_gan(generator, discriminator)
gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='mean_squared_error')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


- Training Function

In [ ]:
def train_gan(epochs, batch_size, sample_interval):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, real_images.shape[0], half_batch)
        real_imgs = real_images[idx]

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_imgs = generator.predict(noise)

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))

        g_loss = gan.train_on_batch(noise, valid_labels)

        if isinstance(g_loss, (list, np.ndarray)):
          g_loss_value = g_loss[0] if len(g_loss) > 0 else g_loss
        else:
            g_loss_value = g_loss
        print(f"{epoch}/{epochs} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss_value:.4f}]")

- Model Training and Visualization

In [ ]:
train_gan(epochs, batch_size, sample_interval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0/100 [D loss: 0.2540, acc.: 42.19%] [G loss: 0.2527]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


1/100 [D loss: 0.2557, acc.: 34.38%] [G loss: 0.2549]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/100 [D loss: 0.2577, acc.: 28.02%] [G loss: 0.2570]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
3/100 [D loss: 0.2579, acc.: 25.11%] [G loss: 0.2574]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
4/100 [D loss: 0.2586, acc.: 21.77%] [G loss: 0.2582]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
5/100 [D loss: 0.2583, acc.: 21.24%] [G loss: 0.2580]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/100 [D loss: 0.2588, acc.: 19.01%] [G loss: 0.2585]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
7/100 [D loss: 0.2581, acc.: 18.57%] [G loss: 0.2579]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
8/100 [D loss: 0.2586, acc.: 17.51%] [G loss: 0.2584]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
9/100 [D loss: 0.2586, acc.: 17.00%] [G loss: 0.2585]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
10/100 [D loss: 0.2585, acc.: 17.16%] [G loss: 0.2583]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
11/100 [D loss: 0.2583, acc.: 17.03%] [G loss: 0.2582]
1/1 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# This is not giving proper output

# sample_images()

# Because of lack of training resources

<hr>
<hr>